In [1]:
#Import required libraries
import gspread
import sys
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe

In [2]:
#Import project specific functions
from column_map import column_map
from yesno_functions import *

In [3]:
#Import shared functions
sys.path.append('../..')
from IPM_Shared_Code_public.Python.google_creds_functions import create_assertion_session
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *
from IPM_Shared_Code_public.Python.sql_functions import *

C:\ProgramData\Anaconda3\lib\site-packages\authlib\client\__init__.py:12: AuthlibDeprecationWarning: Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.
It will be compatible before version 1.0.
Read more <https://git.io/Jeclj#file-rn-md>
  deprecate('Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.', '1.0', 'Jeclj', 'rn')


### Use the config file to setup connections

In [4]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [5]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Execute the function to get the renamed columns for this sheet

In [6]:
#Call the column map function to get the dictionary to be used for renaming and subsetting the columns
col_rename = column_map('ambassador_cw')

In [7]:
#Because of duplicate column names these columns are renamed based on the column index and the keys and 
#values need to be switched
col_rename = {v[0]: k for k, v in col_rename.items()}

In [8]:
cols = list(col_rename.values())

### Read the site reference list from SQL

In [9]:
sql = 'select * from crowdsdb.dbo.tbl_ref_sites'

In [10]:
site_ref = pd.read_sql(con = engine, sql = sql)[['site_id', 'site_desc', 'borough']]

### Read the current data from SQL

In [11]:
sql = 'select * from crowdsdb.dbo.tbl_cw_ambassador'

In [12]:
ambass_sql = (pd.read_sql(con = engine, sql = sql)
              .drop(columns = ['ambassador_id', 'patroncount'])
              .fillna(value = np.nan, axis = 1))

In [13]:
sql_cols = list(ambass_sql.columns.values)

In [14]:
hash_rows(ambass_sql, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

### Read the latest data from Google Sheets

In [15]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(cred_file, scope)
client = gspread.authorize(creds)

In [16]:
sheet = client.open('_COMBINED Inter-Agency Parks Social Distancing Education Ambassador Report (Responses)')

SpreadsheetNotFound: 

In [39]:
ws = sheet.worksheet('RESPONSES')

In [40]:
ambass = (get_as_dataframe(ws, evaluate_formulas = True, header= None)
          #Always remove row 0 with the column headers
          .iloc[1:]
          .rename(columns = col_rename)
          .fillna(value = np.nan, axis = 1))[cols]

In [41]:
ambass.head()

,encounter_timestamp,encounter_datetime,site_desc,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
1,5/27/2020 17:26:24,5/25/2020 17:25:00,Chelsea Park,NaN,Recreation,Rich,Martinez,Christopher,Epps,Bobby,...,NaN,NaN,NaN,Adult fitness equipment,2.0,Yes,Yes,No,NaN,Manhattan
2,5/27/2020 16:52:09,5/27/2020 16:34:00,St Mary's Park-St Mary's Playground West,NaN,Recreation,Dejon,Williams,Jabor,Forde,NaN,...,NaN,NaN,NaN,Playground,42.0,Yes,No,No,patrons cursed and told us to leave the kids t...,Bronx
3,5/27/2020 16:33:53,5/27/2020 16:30:00,Thomas Jefferson Park-Thomas Jefferson Park,NaN,Recreation,Adrienna,Broodie,Kiara,Harris,NaN,...,Track/field area,No,Some Social Distancing guidelines were followe...,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
4,5/27/2020 16:29:08,5/27/2020 3:15:00,Lost Battalion Hall Recreation Center,In the park,Recreation,Chris,Wong,NaN,NaN,NaN,...,NaN,NaN,NaN,Playground,10.0,Yes,Yes,No,NaN,Queens
5,5/27/2020 16:28:04,5/27/2020 16:15:00,Williamsbridge Oval,NaN,Recreation,Sarah,Bishow-Semevolo,NaN,NaN,NaN,...,Track/field area,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx


In [42]:
yesno = ['sd_pdcontact', 'closed_approach', 'closed_outcome', 'closed_pdcontact']

In [43]:
yesno_cols(ambass, yesno)

In [44]:
#Remove rows with no timestamp because these rows have no data
ambass = ambass[ambass['encounter_timestamp'].notnull()]

In [45]:
ambass = ambass.merge(site_ref, how = 'left', on = ['site_desc', 'borough'])#[sql_cols]

In [46]:
ambass.head()

,encounter_timestamp,encounter_datetime,site_desc,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_comments,closed_amenity,closed_patroncount,closed_comments,borough,sd_pdcontact,closed_approach,closed_outcome,closed_pdcontact,site_id
0,5/27/2020 17:26:24,5/25/2020 17:25:00,Chelsea Park,NaN,Recreation,Rich,Martinez,Christopher,Epps,Bobby,...,NaN,Adult fitness equipment,2.0,NaN,Manhattan,NaN,1.0,1.0,0.0,M011
1,5/27/2020 16:52:09,5/27/2020 16:34:00,St Mary's Park-St Mary's Playground West,NaN,Recreation,Dejon,Williams,Jabor,Forde,NaN,...,NaN,Playground,42.0,patrons cursed and told us to leave the kids t...,Bronx,NaN,1.0,0.0,0.0,X045-03
2,5/27/2020 16:33:53,5/27/2020 16:30:00,Thomas Jefferson Park-Thomas Jefferson Park,NaN,Recreation,Adrienna,Broodie,Kiara,Harris,NaN,...,Some Social Distancing guidelines were followe...,NaN,NaN,NaN,Manhattan,0.0,NaN,NaN,NaN,M047-01
3,5/27/2020 16:33:53,5/27/2020 16:30:00,Thomas Jefferson Park-Thomas Jefferson Park,NaN,Recreation,Adrienna,Broodie,Kiara,Harris,NaN,...,Some Social Distancing guidelines were followe...,NaN,NaN,NaN,Manhattan,0.0,NaN,NaN,NaN,M047-03
4,5/27/2020 16:29:08,5/27/2020 3:15:00,Lost Battalion Hall Recreation Center,In the park,Recreation,Chris,Wong,NaN,NaN,NaN,...,NaN,Playground,10.0,NaN,Queens,NaN,1.0,1.0,0.0,Q401


In [50]:
hash_rows(ambass, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

### Find the deltas based on the row hashes

In [51]:
ambass_deltas = (check_deltas(new_df = ambass, old_df = ambass_sql, on = ['encounter_timestamp', 'site_id'], 
                              hash_name = 'row_hash', dml_col = 'dml_verb'))[sql_cols + ['dml_verb']]

In [52]:
ambass_deltas.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough,dml_verb
0,5/27/2020 17:26:24,5/25/2020 17:25:00,M011,NaN,Recreation,Rich,Martinez,Christopher,Epps,Bobby,...,NaN,NaN,Adult fitness equipment,2.0,1.0,1.0,0.0,NaN,Manhattan,I
1,5/27/2020 16:52:09,5/27/2020 16:34:00,X045-03,NaN,Recreation,Dejon,Williams,Jabor,Forde,NaN,...,NaN,NaN,Playground,42.0,1.0,0.0,0.0,patrons cursed and told us to leave the kids t...,Bronx,I
2,5/27/2020 16:33:53,5/27/2020 16:30:00,M047-01,NaN,Recreation,Adrienna,Broodie,Kiara,Harris,NaN,...,0.0,Some Social Distancing guidelines were followe...,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan,I
3,5/27/2020 16:33:53,5/27/2020 16:30:00,M047-03,NaN,Recreation,Adrienna,Broodie,Kiara,Harris,NaN,...,0.0,Some Social Distancing guidelines were followe...,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan,I
4,5/27/2020 16:29:08,5/27/2020 3:15:00,Q401,In the park,Recreation,Chris,Wong,NaN,NaN,NaN,...,NaN,NaN,Playground,10.0,1.0,1.0,0.0,NaN,Queens,I


In [54]:
ambass_inserts = ambass_deltas[ambass_deltas['dml_verb'] == 'I'][sql_cols]

In [ ]:
ambass_inserts.head()

In [55]:
ambass_inserts.to_sql('tbl_cw_ambassador', engine, index = False, if_exists = 'append')

In [56]:
ambass_updates = ambass_deltas[ambass_deltas['dml_verb'] == 'U'][sql_cols]

In [57]:
ambass_updates.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough


In [60]:
sql_update(ambass_updates, 'tbl_cw_ambassador', engine, ['encounter_timestamp', 'site_id'])